In [18]:
from agno.agent import Agent
from agno.workflow import Workflow,StepInput,StepOutput
from pydantic import BaseModel, Field
from agno.models.mistral import MistralChat
from agno.models.google import Gemini
from agno.tools.mcp import MCPTools
from mcp import StdioServerParameters
from textwrap import dedent
from dotenv import load_dotenv
import json
import os

load_dotenv()

True

In [19]:
model=MistralChat()
model2=Gemini()



In [22]:
agent=Agent(
    name="Test Agent",
    description="An agent for testing purposes",
    model=model
    
)

In [23]:
output=await agent.arun("Write a short poem about the sea.")

In [24]:
output.content

'**Ode to the Sea**\n\nOh, restless tide in sapphire hue,\nYou dance and sing in shades of blue.\nWith whispers soft and roars so grand,\nYou shape the shore with gentle hand.\n\nThe gulls above in circles glide,\nWhile waves below in rhythms ride.\nA world of depth, both fierce and free,\nA boundless heart—the mighty sea.'

In [47]:
async def run_agent():

    token=os.getenv('NOTION_API_KEY')
    
    if not token:
        raise ValueError("NOTION_API_KEY not found in environment variables.")
    
    # Set up MCP server parameters
    command = "npx"
    args = ["-y", "@notionhq/notion-mcp-server"]
    env = {
        "OPENAPI_MCP_HEADERS": json.dumps(
            {"Authorization": f"Bearer {token}", "Notion-Version": "2022-06-28"}
        )
    }
    server_params = StdioServerParameters(command=command, args=args, env=env)
    
    async with MCPTools(server_params=server_params) as mcp_tools:
        agent_with_tools = Agent(
            name="Notion Agent",
            description="An agent that can interact with Notion via MCP tools.",
            model=model,
            tools=[mcp_tools]
        )
        
        response = await agent_with_tools.arun("Create a new page in my Notion workspace titled 'Test Page'.")
        
        print(response.content)
    


In [48]:
await run_agent()

It seems I made an incorrect assumption about the `parent` parameter. To create a page in your Notion workspace, I need to know the ID of the parent page or database where you want to create this page.

Here’s how we can proceed:

1. **If you want to create the page under a specific page or database**, provide me with the **parent page ID** or **database ID**.
2. **If you want to create it in the root of your workspace**, I can first search for the root page or use the default workspace ID.

### How to Find a Page or Database ID:
- Open the page or database in Notion.
- The URL will look like: `https://www.notion.so/{workspace_name}/{page_id}` or `https://www.notion.so/{workspace_name}/{database_id}?v=...`.
- The **page_id** or **database_id** is a 32-character UUID (e.g., `12345678-1234-5678-1234-567812345678`).

### Next Steps:
- Share the **parent page ID** or **database ID** where you'd like to create the "Test Page."
- Alternatively, I can search your workspace for existing pages/

In [25]:

async def run_agent(message: str) -> None:
    """Run the GitHub agent with the given message."""

    # Initialize the MCP server
    server_params = StdioServerParameters(
        command="npx",
        args=["-y", "@modelcontextprotocol/server-github"],
        env={
            "GITHUB_PERSONAL_ACCESS_TOKEN": os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN"),
            "PATH": os.getenv("PATH"),  # Required so npx can be found
            "HOME": os.getenv("HOME"),  # Sometimes required for npx caching
        }
    )
    

    # Create a client session to connect to the MCP server
    async with MCPTools(server_params=server_params) as mcp_tools:
        agent = Agent(
            tools=[mcp_tools],
            model=model,
            instructions=dedent("""\
                You are a GitHub assistant. Help users explore repositories and their activity.

                - Use headings to organize your responses
                - Be concise and focus on relevant information\
            """)
                    )

        # Run the agent
        output=await agent.arun(message)
        print(output)

In [26]:

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify the token is loaded
if not os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN"):
    print("Error: GITHUB_PERSONAL_ACCESS_TOKEN is not set.")
    exit(1)


In [50]:
from agno.agent import Agent
from agno.os import AgentOS
from agno.team import Team
from agno.tools.arxiv import ArxivTools
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.hackernews import HackerNewsTools
from agno.tools.newspaper4k import Newspaper4kTools
from agno.tools.wikipedia import WikipediaTools
from agno.tools.youtube import YouTubeTools
from agno.tools.reddit import RedditTools
from agno.tools.gmail import GmailTools
from agno.db.in_memory import InMemoryDb
from agno.tools.local_file_system import LocalFileSystemTools
from dotenv import load_dotenv
from pathlib import Path

# load the api keys into env
load_dotenv()

# define the dir to save research papers
dir_path = Path("./research_papers/")
dir_path.mkdir(exist_ok=True)

# define the dir to save the generated medium articles
target_dir = Path("./medium_articles/")
target_dir.mkdir(exist_ok=True)

# define orchestrator and agent model
orchestrator_model = MistralChat()
model = MistralChat()

# create db for the team leader
db = InMemoryDb()

# ============================= Agents===========================================

# define my sub agents/ members

# define the arxiv research agent
arxiv_research_agent = Agent(
    id="archive-research-agent",
    name="Archive Research Agent",
    model=model,
    role="Arxiv Research Assistant",
    instructions=["You are a research assistant that gathers research papers from Arxiv",
                  "Use the available tools to search for research papers, authors and topics as per user's request",
                  "Summarize your findings clearly and concisely"],
    tools=[ArxivTools(download_dir=dir_path)],
    add_datetime_to_context=True
)


# define the web search agent 
web_search_agent = Agent(
    id="web-search-agent",
    name="Web Search Agent",
    role="Web Research Assistant",
    model=model,
    instructions=["You are a research assistant that gathers information from the web",
                  "Use the available tools to search for articles, summaries and other important material based on the topic the user requested about",
                  "Summarize your findings along with the resource links"],
    add_datetime_to_context=True,
    tools=[DuckDuckGoTools()]
)


# define the hackernews research agent
hackernews_research_agent = Agent(
    id="hackernews-research-agent",
    name="HeckerNews Research Agent",
    model=model,
    role="HackerNews Research Assistant",
    instructions=["You are an expert research assistant that can access HackerNews",
                  "Get relevant information for the recent topics, and get information about the articles for the topic user requested for",
                  "Summarize your findings in proper format"],
    add_datetime_to_context=True,
    tools=[HackerNewsTools()]
)


# define the news article research agent
news_article_research_agent = Agent(
    id="news-article-research-agent",
    name="News Article Research Agent",
    model=model,
    role="News Article Research Agent",
    instructions=["You are a research assistant that can read the contents of articles",
                  "Whenever an url is provided you can read the content of the article and can also get its data",
                  "Using the available tools search for articles and summarize them and gather relevant information"],
    add_datetime_to_context=True,
    tools=[Newspaper4kTools(include_summary=True)]
)


# define the wikipedia tool
wikipedia_research_agent = Agent(
    id="wikipedia-research-agent",
    name="Wikipedia Research Agent",
    role="Wikipedia Research Assistant",
    model=model,
    instructions=["You are a research assistant that gathers information from Wikipedia based on the input topic",
                  "You have the capability to search for articles and gather its content",
                  "Summarize the findings and mention the appropriate resources and references in your output"],
    add_datetime_to_context=True,
    tools=[WikipediaTools()]
)

# youtube research agent
youtube_research_agent = Agent(
    id="youtube-research-agent",
    name="Youtube Research Agent",
    model=model,
    role="Youtube Research Assistant",
    instructions=["You are a research assistant that gathers information from Youtube",
                  "You have the capability to read youtube video transcripts and summarize them",
                  "You can also read metadata related to youtube videos",
                  "you can also fetch timestamps of a particular video",
                  "summarize the transcripts in clear and concised manner"],
    add_datetime_to_context=True,
    tools=[YouTubeTools()]
)



# gmail agent
gmail_agent = Agent(
    id="gmail-agent",
    name="Gmail Agent",
    role="Manages mails through Gmail",
    instructions=["You are a helpful agent that can draft messages using gmail",
                  "you have the capability to draft mails, read mails and send mails whenever requested by the user",
                  "you have the capability to search for mails and read them",
                  "make sure to always confirm the drafted before sending it to the mail id",
                  "make sure to write the mails in proper format including a relevant subject line"],
    model=model,
    add_datetime_to_context=True,
    tools=[GmailTools(port=8000)]
)


# ======================== Team =================================


medium_article_team = Team(
    id="medium-article-creation-team",
    name="Medium Article Creation Team",
    role="Team Leader which manages research and content creation",
    db=db,
    members=[arxiv_research_agent,
             web_search_agent,
             hackernews_research_agent,
             news_article_research_agent,
             wikipedia_research_agent,
             youtube_research_agent,
             gmail_agent],
    model=orchestrator_model,
    instructions=["You are a team leader managing multiple sub agents in your team",
                  "You have access to agents which can do research based on the topic on various sources such as arxiv, X(formerly twitter), youtube, reddit, wikipedia, hackernews, newspaper articles, web search using google search and duckduckgo search",
                  "you also have the capability to read and write emails",
                  "your task is to understand the topic given by the user and fetch relevant research information using your team members",
                  "once you have enough research material, your primary task is to create medium(platform) styled articles",
                  "for checking how articles are written on medium you can use the article research agent",
                  "once you have created the medium article, show the user the final draft",
                  "only when the user confirms the draft, save it to the filesystem in a markdown format as a .md file using the filename suggested by user",
                  "if the user does not give a filename, then use a self created name based on the topic on which the article was created"],
    add_datetime_to_context=True,
    add_history_to_context=True,
    num_history_runs=10,
    tools=[LocalFileSystemTools(target_directory=target_dir,
                                default_extension="md")],
    markdown=True
)

# ======================== AgentOS ============================


In [51]:
await medium_article_team.arun('topic:"Pytorch')

Tried calling set_status on an ended span.
Setting attribute on ended span.
Calling end() on an ended span.
Tried calling set_status on an ended span.
Setting attribute on ended span.
Calling end() on an ended span.


INFO Searching wikipedia for: PyTorch

INFO Searching wikipedia for: Comparison of deep learning frameworks

INFO Searching wikipedia for: History of PyTorch

Tried calling set_status on an ended span.
Setting attribute on ended span.
Calling end() on an ended span.


...

INFO Searching wikipedia for: PyTorch vs TensorFlow

INFO Searching wikipedia for: Key features of PyTorch

INFO Searching wikipedia for: PyTorch documentation

Tried calling set_status on an ended span.
Setting attribute on ended span.
Calling end() on an ended span.
Tried calling set_status on an ended span.
Setting attribute on ended span.
Calling end() on an ended span.
Tried calling set_status on an ended span.
Setting attribute on ended span.
Calling end() on an ended span.


TeamRunOutput(run_id='c0b266be-767e-4240-9394-07560c98a64c', team_id='medium-article-creation-team', team_name='Medium Article Creation Team', session_id='96dc8a82-d507-4798-ae28-402f15c58234', parent_run_id=None, user_id=None, input=TeamRunInput(input_content='topic:"Pytorch', images=None, videos=None, audios=None, files=None), content='To create a high-quality Medium-style article on **"PyTorch"**, I will follow these steps:\n\n---\n\n### **Step 1: Research Phase**\nI will delegate tasks to the following team members to gather comprehensive information about PyTorch:\n1. **Archive Research Agent**: Fetch recent and foundational research papers on PyTorch from arXiv.\n2. **Wikipedia Research Agent**: Retrieve a concise overview of PyTorch, its history, and its key features.\n3. **Web Search Agent**: Gather the latest tutorials, blogs, and official documentation about PyTorch.\n4. **HackerNews Research Agent**: Find discussions, trends, and community insights about PyTorch.\n5. **News 